# Grib2 exploration

In [ ]:
from osgeo import gdal
import re
import pandas as pd

class GribFileInfo:
    def __init__(self,file:str):
        self.file = file
        pass

    def to_pandas(self):
        src_ds = gdal.Open(self.file)
        pass
        if src_ds is not None: 
            info = gdal.Info(src_ds)
            info = info.replace('Band','+++Band')
            # display(info)
            pos = info.find('Band')
            if pos != -1:
                info = info[pos:]
            # info = info.replace('\n','')
            info = info.split('+++')    
            dicts = []
            for band in info:
                a = re.split('\n',band)
                d = {}
                for b in a:
                    i = b.split('=')
                    # display(i)
                    # display(len(i))
                    if len(i) > 1:
                        k = i[0].strip().replace(' ','_')
                        if 'Band' in k:
                            k = 'Band'
                        v = ''.join(i[1:]).strip() if len(i[1:]) > 1 else i[1].strip()
                        d[k] = v
                dicts.append(d)

        df = pd.DataFrame(dicts)
        df['Band_number'] = range(1,len(df.index)+1)
        # df.loc[df.GRIB_COMMENT.str.contains('Temperature')]
        # display(type(df.columns))

        col1 = [df.columns[-1]]
        cols = list(df.columns)[1:-1]
        # display(col1 + cols)
        df = df[col1 + cols]
        return df

df = GribFileInfo('/fs/site5/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/2019010100_000.grib2').to_pandas()

# get temperature bands
display(df.loc[df.GRIB_COMMENT.str.contains('Temperature')])
display(df.loc[df.GRIB_COMMENT.str.contains('Temperature')].Band_number.tolist())

# display(d)
# info = info.split('Band')
# for band in info:
#     band
    

In [ ]:
import os
import re
import xarray as xr
import numpy as np
import fstpy.all as fstpy
import pandas as pd
path = '/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/'
files = np.sort([path + f for f in os.listdir(path) if re.search(r'[0-9]{10}_[0-9]{3}.grib2', f)])

filters = [
    {'typeOfLevel': 'surface'},
    {'typeOfLevel': 'meanSea'},
    {'typeOfLevel': 'nominalTop'},
    {'typeOfLevel': 'atmosphere'},
    {'typeOfLevel': 'eta'},
    {'stepType': 'instant', 'typeOfLevel': 'heightAboveGround'},
    {'stepType': 'min', 'typeOfLevel': 'heightAboveGround'},
    {'stepType': 'max', 'typeOfLevel': 'heightAboveGround'},
    {'typeOfLevel': 'depthBelowLandLayer'},
    {'typeOfLevel': 'isobaricInhPa'},
    {'typeOfLevel': 'isobaricLayer'},
]
dicts = []
for current_file in files:
    for filter in filters:
        # print(filter)
        try:
            ds = xr.open_dataset(current_file, engine="cfgrib",filter_by_keys=filter)
            if 'stepType' in filter.keys():
                step_type = filter['stepType']
            else:
                step_type = ''
            # display(ds)
            dicts = dicts + [{'file':current_file[-20:],'time':pd.to_datetime(ds.time.data).strftime('%Y:%m:%d-%H:%M:%S'), 'name':ds[i].GRIB_name, 'units':ds[i].GRIB_units, 'step_type':step_type, 'type_of_level':ds[i].GRIB_typeOfLevel} for i in ds.data_vars]
        except:
            pass
    break    

[fstpy.delete_file(path + f) for f in os.listdir(path) if re.search(r'.*.idx', f)]

df = pd.DataFrame(dicts)
# display(df)
fstpy.delete_file('/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/grib2_file_contents.csv')
df.to_csv('/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/grib2_file_contents.csv', index=False)


In [ ]:
pd.read_csv('/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/grib2_file_contents.csv')

In [ ]:
fstpy.delete_file('/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/grib2_unique_variables.csv')
df[['name','units','step_type','type_of_level']].drop_duplicates().to_csv('/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/grib2_unique_variables.csv', index=False)
pd.read_csv('/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/grib2_unique_variables.csv')

In [ ]:
mod_df = pd.read_csv('/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/grib2_file_contents.csv')
mod_df['level'] = None
mod_df.loc[(mod_df.type_of_level=='heightAboveGround') & (mod_df.name == '10 metre U wind component'), 'level'] = '10'
mod_df.loc[(mod_df.type_of_level=='heightAboveGround') & (mod_df.name == '10 metre V wind component'), 'level'] = '10'
mod_df.loc[(mod_df.type_of_level=='heightAboveGround') & (mod_df.name == '10 metre wind speed'), 'level'] = '10'
mod_df.loc[(mod_df.type_of_level=='heightAboveGround') & (mod_df.name == '10 metre wind direction'), 'level'] = '10'

print(mod_df.to_string())

In [ ]:
import os
import re
import xarray as xr
import numpy as np
import fstpy.all as fstpy
import pandas as pd
path = '/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/'
files = np.sort([path + f for f in os.listdir(path) if re.search(r'[0-9]{10}_[0-9]{3}.grib2', f)])

filters = [
    {'typeOfLevel': 'surface'},
    {'typeOfLevel': 'meanSea'},
    {'typeOfLevel': 'nominalTop'},
    {'typeOfLevel': 'atmosphere'},
    {'typeOfLevel': 'eta'},
    {'stepType': 'instant', 'typeOfLevel': 'heightAboveGround'},
    {'stepType': 'min', 'typeOfLevel': 'heightAboveGround'},
    {'stepType': 'max', 'typeOfLevel': 'heightAboveGround'},
    {'typeOfLevel': 'depthBelowLandLayer'},
    {'typeOfLevel': 'isobaricInhPa'},
    {'typeOfLevel': 'isobaricLayer'},
]

def get_levels(ds, filter):
    if filter['typeOfLevel'] == 'eta':
        return ','.join(ds.eta.data.astype(str).tolist())
    if filter['typeOfLevel'] == 'isobaricInhPa':
        return ','.join(ds.isobaricInhPa.data.astype(str).tolist())
    return ''


ds = xr.open_dataset(files[0], engine="cfgrib",filter_by_keys=filters[9])
display(ds)
# get_levels(ds,filters[9])


In [ ]:
ds.r.sel(isobaricInhPa=1000).plot(size=15,cmap='jet')

In [ ]:
import os
import re
import xarray as xr
import numpy as np
import fstpy.all as fstpy
import pandas as pd
path = '/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/'
files = np.sort([path + f for f in os.listdir(path) if re.search(r'[0-9]{10}_[0-9]{3}.grib2', f)])

filters = [
    {'typeOfLevel': 'surface'},
    {'typeOfLevel': 'meanSea'},
    {'typeOfLevel': 'nominalTop'},
    {'typeOfLevel': 'atmosphere'},
    {'typeOfLevel': 'eta'},
    {'stepType': 'instant', 'typeOfLevel': 'heightAboveGround'},
    {'stepType': 'min', 'typeOfLevel': 'heightAboveGround'},
    {'stepType': 'max', 'typeOfLevel': 'heightAboveGround'},
    {'typeOfLevel': 'depthBelowLandLayer'},
    {'typeOfLevel': 'isobaricInhPa'},
    {'typeOfLevel': 'isobaricLayer'},
]


def get_levels(ds, filter):
    if filter['typeOfLevel'] == 'eta':
        return ','.join(ds.eta.data.astype(str).tolist())
    if filter['typeOfLevel'] == 'isobaricInhPa':
        return ','.join(ds.isobaricInhPa.data.astype(str).tolist())
    return ''

dicts = []
for current_file in files:
    for filter in filters:
        # print(filter)
        try:
            ds = xr.open_dataset(current_file, engine="cfgrib",filter_by_keys=filter)
            # display(ds)
            dicts = dicts + [{'file':current_file[-20:],'time':pd.to_datetime(ds.time.data).strftime('%Y:%m:%d-%H:%M:%S'), 'var':i, 'name':ds[i].GRIB_name, 'units':ds[i].GRIB_units, 'type_of_level':ds[i].GRIB_typeOfLevel,'levels':get_levels(ds,filter)} for i in ds.data_vars]
        except:
            pass

[fstpy.delete_file(path + f) for f in os.listdir(path) if re.search(r'.*.idx', f)]

df = pd.DataFrame(dicts)
# display(df)
fstpy.delete_file('/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/grib2_file_contents_with_levels.csv')
df.to_csv('/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/grib2_file_contents_with_levels.csv', index=False)


In [ ]:
pd.set_option('display.max_rows', 500)
df = pd.read_csv('/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/grib2_file_contents_with_levels.csv')
df

In [ ]:
import xarray as xr
from cfgrib import xarray_store
from cfgrib import xarray_to_grib
import cfgrib
display(cfgrib.__version__)
# dst = xarray_store.open_dataset("/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/2019010100_000.grib2")
# ds = xr.open_dataset("/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/2019010100_000.grib2", engine="cfgrib",filter_by_keys={'stepType': 'instant', 'typeOfLevel': 'heightAboveGround','cfVarName':'t2m'})
# ds

# cfgrib.to_grib(ds, '/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/2019010100_000_t2m.grib2')#, grib_keys={'centre': 'ecmf'})
# ds = xr.open_dataset("/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/tests/2021010100_000.grib2", engine="cfgrib")
# ds.to_netcdf("/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/tests/2t_2021010100_000instant_heightAboveGround.nc")
# ds

In [ ]:
# from cfgrib.xarray_to_grib import to_grib
# del ds['time']
# del ds['step']
# del ds['heightAboveGround']
# del ds['valid_time']
# ds.t2m
# to_grib(ds.t2m, '/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/2019010100_000_t2m.grib2', grib_keys={'edition': 2})

# gdal_translate -b 30 2019010100_000.grib2 2mt_2019010100_000.tiff
# gdalwarp -t_srs EPSG:4326 2mt_2019010100_000.tiff 2mt_2019010100_000_EPSG4326.tiff

In [ ]:
from pathlib import Path
from osgeo import gdal
import glob
import datetime
import re
import copy
import numpy as np
path = Path('/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/split')
file = '2017010100_000.grib2_99.tiff'
files = glob.glob(str(path / '*grib2_*'))
# display(len(files))
def mult_sub(trans,text):
    # Create a regular expression  from the dictionary keys
    regex = re.compile("(%s)" % "|".join(map(re.escape, trans.keys())))
    # For each match, look-up corresponding value in dictionary
    return regex.sub(lambda mo: trans[mo.string[mo.start():mo.end()]], text) 

def make_tiff(grib_file):
    ds = gdal.Warp(str(file))

flist = []
for f in files:
    ds = gdal.Open(str(f))
    rb = ds.GetRasterBand(1)
    desc = rb.GetDescription()
    level = copy.copy(desc)
    type_of_level = copy.copy(desc)
    level = re.sub('([0-9]+).*? .*?=.*$',r'\1',level)
    # display(level)
    type_of_level = re.sub('.*? (.*?)=.*$',r'\1',type_of_level)
    if 'ISBL' == type_of_level:
        level = int(level)//100
        level = f'{level:04d}'

    # level = int(level)//100
    # display(rb.GetMetadata())
    filename = f"CMC_hrdps_continental_{rb.GetMetadata()['GRIB_ELEMENT']}_{type_of_level}_{level}_ps2.5km_{datetime.datetime.fromtimestamp(int(rb.GetMetadata()['GRIB_VALID_TIME'])).strftime('%Y%m%d%H')}_P{int(rb.GetMetadata()['GRIB_FORECAST_SECONDS'])//3600:03d}-00"
    filename = filename.replace(' ','')
    flist.append(filename)
np.sort(flist)
    

In [ ]:
import subprocess
import logging
from pathlib import Path
from osgeo import gdal
import glob
import copy
import re 
import datetime

logger = logging.getLogger()

def setenv(key:str, value:str) -> None:
    
    if not (os.environ.get(key) is None):
        os.environ[key] = ''.join([os.environ[key], ':', value])
    else:
        os.environ[key] = value

def split_grib(file: str) -> str:
    file_only = os.path.basename(file)
    process = subprocess.Popen(['grib2_split_file', f'{file}'],
                        stdout=subprocess.PIPE, 
                        stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    _ = process.wait()
    stdout = stdout.decode('utf-8')
    stderr = stderr.decode('utf-8')
    if stdout == '':    
        logger.info(stdout)
    if stderr == '':
        logger.error(stderr)
        exit
    logger.info(f'Succesfully processed {file_only} !')
    return file_only

def rename_grib_files(original_file:str, search_pattern:str='*grib2_*') -> None:
    file_only = os.path.basename(original_file)
    # display(file_only)
    extension = os.path.splitext(file_only)[1]
    # display(extension)
    path_only = original_file.replace(file_only,'')
    # display(path_only)
    files = glob.glob(str('/'.join([path_only, search_pattern])))
    # display(files)
    for old_file in files:
        ds = gdal.Open(str(old_file))
        rb = ds.GetRasterBand(1)
        desc = rb.GetDescription()
        level = copy.copy(desc)
        type_of_level = copy.copy(desc)
        level = re.sub('([0-9]+).*? .*?=.*$',r'\1',level)
        # display(level)
        type_of_level = re.sub('.*? (.*?)=.*$',r'\1',type_of_level)
        if 'ISBL' == type_of_level:
            level = int(level)//100
            level = f'{level:04d}'

        # level = int(level)//100
        # display(rb.GetMetadata())
        new_file = f"{path_only}CMC_hrdps_continental_{rb.GetMetadata()['GRIB_ELEMENT']}_{type_of_level}_{level}_ps2.5km_{datetime.datetime.fromtimestamp(int(rb.GetMetadata()['GRIB_VALID_TIME'])).strftime('%Y%m%d%H')}_P{int(rb.GetMetadata()['GRIB_FORECAST_SECONDS'])//3600:03d}-00{extension}"
        new_file = new_file.replace(' ','')
        os.rename(old_file,new_file)
        # display(new_file)


setenv('LD_LIBRARY_PATH', '/fs/ssm/eccc/cmd/cmds/ext/20220202/rhel-8-amd64-64/lib')
setenv('PATH', '/fs/ssm/eccc/cmd/cmds/apps/grib/grib2-cvt/2.3/rhel-8-amd64-64/bin/')
setenv('GRIB2_CVT_HOME', '/fs/ssm/eccc/cmd/cmds/apps/grib/grib2-cvt/2.3/grib2_2.3_rhel-8-amd64-64')
setenv('ECCODES_DEFINITION_PATH', '/fs/ssm/eccc/cmd/cmds/ext/master/eccodes_2.24.2-2_rhel-8-amd64-64/share/eccodes/definitions')
setenv('ECCODES_SAMPLES_PATH', '/fs/ssm/eccc/cmd/cmds/ext/master/eccodes_2.24.2-2_rhel-8-amd64-64/share/eccodes/samples')

path = Path('/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/test')
filename = '2020010100_003.grib2'
file = split_grib(str(path / filename))
rename_grib_files(str(path / filename))



# Get unique variables

In [ ]:
import subprocess
import logging
from pathlib import Path
from osgeo import gdal
import glob
import copy
import re 
import datetime
import numpy as np
logger = logging.getLogger()

def setenv(key:str, value:str) -> None:
    
    if not (os.environ.get(key) is None):
        os.environ[key] = ''.join([os.environ[key], ':', value])
    else:
        os.environ[key] = value

def split_grib(file: str) -> str:
    file_only = os.path.basename(file)
    process = subprocess.Popen(['grib2_split_file', f'{file}'],
                        stdout=subprocess.PIPE, 
                        stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    _ = process.wait()
    stdout = stdout.decode('utf-8')
    stderr = stderr.decode('utf-8')
    if stdout == '':    
        logger.info(stdout)
    if stderr == '':
        logger.error(stderr)
        exit
    logger.info(f'Succesfully processed {file_only} !')
    return file_only

def rename_grib_files(original_file:str, search_pattern:str='*grib2_*') -> None:
    file_only = os.path.basename(original_file)
    # display(file_only)
    extension = os.path.splitext(file_only)[1]
    # display(extension)
    path_only = original_file.replace(file_only,'')
    # display(path_only)
    files = glob.glob(str('/'.join([path_only, search_pattern])))
    # display(files)
    for old_file in files:
        ds = gdal.Open(str(old_file))
        rb = ds.GetRasterBand(1)
        desc = rb.GetDescription()
        level = copy.copy(desc)
        type_of_level = copy.copy(desc)
        level = re.sub('([0-9]+).*? .*?=.*$',r'\1',level)
        # display(level)
        type_of_level = re.sub('.*? (.*?)=.*$',r'\1',type_of_level)
        if 'ISBL' == type_of_level:
            level = int(level)//100
            level = f'{level:04d}'

        # level = int(level)//100
        # display(rb.GetMetadata())
        new_file = f"{path_only}CMC_hrdps_continental_{rb.GetMetadata()['GRIB_ELEMENT']}_{type_of_level}_{level}_ps2.5km_{datetime.datetime.fromtimestamp(int(rb.GetMetadata()['GRIB_VALID_TIME'])).strftime('%Y%m%d%H')}_P{int(rb.GetMetadata()['GRIB_FORECAST_SECONDS'])//3600:03d}-00{extension}"
        new_file = new_file.replace(' ','')
        os.rename(old_file,new_file)
        # display(new_file)



path = Path('/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/')
files1 = glob.glob(str('/'.join([str(path), '*010100_000.grib2'])))
files2 = glob.glob(str('/'.join([str(path), '*060100_000.grib2'])))
files3 = glob.glob(str('/'.join([str(path), '*120100_000.grib2'])))
files4 = glob.glob(str('/'.join([str(path), '*011500_000.grib2'])))
files5 = glob.glob(str('/'.join([str(path), '*061500_000.grib2'])))
files6 = glob.glob(str('/'.join([str(path), '*121500_000.grib2'])))
files = np.sort(files1 + files2 + files3 + files4 + files5 + files6)

variables = []

for f in files:
    print(f'Processing {f}')
    ds = gdal.Open(str(f))

    for i in range(1,ds.RasterCount+1):
        rb = ds.GetRasterBand(i)
        meta = rb.GetMetadata()
        variables.append(f"{meta['GRIB_ELEMENT']},{meta['GRIB_COMMENT']},{meta['GRIB_SHORT_NAME']},{meta['GRIB_UNIT']},{rb.GetDescription()}\n")

variables = np.sort(np.unique(variables))
variables = np.insert(variables,0,'GRIB_ELEMENT,GRIB_COMMENT,GRIB_SHORT_NAME,GRIB_UNIT,DESCRIPTION\n')
outfile = str(path / 'unique_vars.csv')

if os.path.isfile(outfile):
    os.remove(outfile)

with open(outfile,'w+') as fp:
    fp.writelines(variables)


In [ ]:
import pandas as pd
pd.options.display.max_rows = 999
df = pd.read_csv('/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/unique_vars.csv')

print(df.GRIB_COMMENT.to_string(index=False))
# variables = {}
# for i in df.index:
#     variables[] = count

# variables

In [ ]:
with open('layer_meta/parameters','r') as fp:
    a = fp.readlines()

new_list = []
for item in a:
    item = item.replace('\n','')
    new_list.append(f"'{item}',\n")

with open('layer_meta/parameters2','w') as fp:
    fp.writelines(new_list)

new_list

In [ ]:
import glob
from pathlib import Path
import numpy as np
import shutil

year = '2021'
path = Path(f'/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/{year}')

files = np.sort(glob.glob(str(path / f'{year}*')))

# display(files)

def create_dir(dir):
    print(f'creaing directory {dir}')
    Path(dir).mkdir(parents=True, exist_ok=True)

def copy_file(orig,dest):
    print(f'copy {orig} -> {dest}')
    shutil.copyfile(orig, dest)

# display(os.path.abspath(files[0]))

for f in files:
    fname = os.path.basename(f)
    dname = os.path.dirname(f)
    month = fname[4:6]
    day = fname[6:8]
    hh = fname[8:10]
    p = fname[11:14]
    dir = f'{dname}/{month}/{day}/{hh}/{p}'
    create_dir(dir)
    print(f'{f} -> {dir}/{fname}')

    shutil.copyfile(f, f'{dir}/{fname}')
    # print(f'{month}/{day}/{hh}/{p}')
    # break


In [ ]:
import pandas as pd
import xarray as xr
import numpy as np

# Column 1:  Date, in the format YYYY-MM-DD
# Column 2:  Time, in GMT, given to nanosecond resolution
# Column 3:  Latitude, in signed decimal degrees
# Column 4:  Longitude, in signed decimal degrees
# Column 5:  Event strength, in kiloamperes
# Column 6:  Chi squared value
# Column 7:  Semi-major axis of the 50% confidence ellipse, in kilometers
# Column 8:  Semi-minor axis of the 50% confidence ellipse, in kilometers
# Column 9:  Angle of the confidence ellipse from North
# Column 10: Multiplicity of the event

df = pd.read_csv('199801f.csv')

convert_dict = {
    'date':np.datetime64, 
    'lat':np.float32, 
    'lon':np.float32, 
    'strength':np.float32, 
    'chi_sq_val':np.float32, 
    'semi_maj_axis':np.float32,
    'semi_min_axis':np.float32, 
    'angle':np.int32, 
    'mult':np.int32, 
    'status':str,
            }

df = df.astype(convert_dict)

df['time'] = df.date
df = df.drop(columns=['status','date'])
display(df)
# df.to_xarray()
# df

# df.to_xarray()
import geopandas as gpd
gdf = gpd.GeoDataFrame(
    df['strength'], geometry=gpd.points_from_xy(df.lon, df.lat)
)
gdf.crs = 'epsg:4326'
gdf.plot(figsize=(20,20))

In [ ]:
from osgeo import gdal
import os
import logging
import glob
import subprocess
import copy
import re
import datetime
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

def setenv(key:str, value:str) -> None:
    
    if not (os.environ.get(key) is None):
        if value not in os.environ[key]:
            os.environ[key] = ''.join([os.environ[key], ':', value])
    else:
            os.environ[key] = value

def set_vars():
    setenv('LD_LIBRARY_PATH', '/fs/ssm/eccc/cmd/cmds/ext/20220202/rhel-8-amd64-64/lib')
    setenv('PATH', '/fs/ssm/eccc/cmd/cmds/apps/grib/grib2-cvt/2.3/rhel-8-amd64-64/bin/')
    setenv('GRIB2_CVT_HOME', '/fs/ssm/eccc/cmd/cmds/apps/grib/grib2-cvt/2.3/grib2_2.3_rhel-8-amd64-64')
    setenv('ECCODES_DEFINITION_PATH', '/fs/ssm/eccc/cmd/cmds/ext/master/eccodes_2.24.2-2_rhel-8-amd64-64/share/eccodes/definitions')
    setenv('ECCODES_SAMPLES_PATH', '/fs/ssm/eccc/cmd/cmds/ext/master/eccodes_2.24.2-2_rhel-8-amd64-64/share/eccodes/samples')
    logger.info('Setting environment variables for  grib2_split_file')
    logger.debug(f"LD_LIBRARY_PATH={os.environ['LD_LIBRARY_PATH']}")
    logger.debug(f"PATH={os.environ['PATH']}")
    logger.debug(f"GRIB2_CVT_HOME={os.environ['GRIB2_CVT_HOME']}")
    logger.debug(f"ECCODES_DEFINITION_PATH={os.environ['ECCODES_DEFINITION_PATH']}")
    logger.debug(f"ECCODES_SAMPLES_PATH={os.environ['ECCODES_SAMPLES_PATH']}")
    
set_vars()

def split_grib(file: str) -> None:
    file_only = os.path.basename(file)
    process = subprocess.Popen(['grib2_split_file', f'{file}'],
                        stdout=subprocess.PIPE, 
                        stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    _ = process.wait()
    stdout = stdout.decode('utf-8')
    stderr = stderr.decode('utf-8')
    if stdout != '':    
        logger.info(stdout)
    if stderr != '':
        logger.error(stderr)
        exit
    logger.info(f'Succesfully processed {file_only} !')

# Create a WarpOptions() object that can be passed to gdal.Warp()
# Keyword arguments are :
#   options --- can be be an array of strings, a string or let empty and filled from other keywords.
#   format --- output format ("GTiff", etc...)
#   outputBounds --- output bounds as (minX, minY, maxX, maxY) in target SRS
#   outputBoundsSRS --- SRS in which output bounds are expressed, in the case they are not expressed in dstSRS
#   xRes, yRes --- output resolution in target SRS
#   targetAlignedPixels --- whether to force output bounds to be multiple of output resolution
#   width --- width of the output raster in pixel
#   height --- height of the output raster in pixel
#   srcSRS --- source SRS
#   dstSRS --- output SRS
#   coordinateOperation -- coordinate operation as a PROJ string or WKT string
#   srcAlpha --- whether to force the last band of the input dataset to be considered as an alpha band
#   dstAlpha --- whether to force the creation of an output alpha band
#   outputType --- output type (gdalconst.GDT_Byte, etc...)
#   workingType --- working type (gdalconst.GDT_Byte, etc...)
#   warpOptions --- list of warping options
#   errorThreshold --- error threshold for approximation transformer (in pixels)
#   warpMemoryLimit --- size of working buffer in MB
#   resampleAlg --- resampling mode
#   creationOptions --- list of creation options
#   srcNodata --- source nodata value(s)
#   dstNodata --- output nodata value(s)
#   multithread --- whether to multithread computation and I/O operations
#   tps --- whether to use Thin Plate Spline GCP transformer
#   rpc --- whether to use RPC transformer
#   geoloc --- whether to use GeoLocation array transformer
#   polynomialOrder --- order of polynomial GCP interpolation
#   transformerOptions --- list of transformer options
#   cutlineDSName --- cutline dataset name
#   cutlineLayer --- cutline layer name
#   cutlineWhere --- cutline WHERE clause
#   cutlineSQL --- cutline SQL statement
#   cutlineBlend --- cutline blend distance in pixels
#   cropToCutline --- whether to use cutline extent for output bounds
#   copyMetadata --- whether to copy source metadata
#   metadataConflictValue --- metadata data conflict value
#   setColorInterpretation --- whether to force color interpretation of input bands to output bands
#   overviewLevel --- To specify which overview level of source files must be used
#   callback --- callback method
#   callback_data --- user data for callback


base_path = '/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/'
files = ['/'.join([base_path, '2020120700_000.grib2'])]


def get_layer_name(path_only,ds) -> str:
    extension = '.tif'
    rb = ds.GetRasterBand(1)
    desc = rb.GetDescription()
    level = copy.copy(desc)
    type_of_level = copy.copy(desc)
    level = re.sub('([0-9]+).*? .*?=.*$',r'\1',level)
    # display(level)
    type_of_level = re.sub('.*? (.*?)=.*$',r'\1',type_of_level)
    if 'ISBL' == type_of_level:
        level = int(level)//100
        level = f'{level:04d}'

    # level = int(level)//100
    # display(rb.GetMetadata())
    new_file = f"{path_only}CMC_hrdps_continental_{rb.GetMetadata()['GRIB_ELEMENT']}_{type_of_level}_{level}_EPSG_4326_{datetime.datetime.fromtimestamp(int(rb.GetMetadata()['GRIB_VALID_TIME'])).strftime('%Y%m%d%H')}_P{int(rb.GetMetadata()['GRIB_FORECAST_SECONDS'])//3600:03d}-00{extension}"
    new_file = new_file.replace(' ','')
    return new_file

for grib_file in files:
    print(f'Processing {grib_file}')
    split_grib(grib_file)
    split_files = glob.glob('/'.join([base_path,'2020120700_000.grib2_*']))
    # for file in split_files:
    for file in [grib_file]:
        print(f'Processing {file}')
        ds = gdal.Open(file)
        filename = get_layer_name('/home/sbf000/ss6/',ds)
        warp_options = gdal.WarpOptions(dstSRS='EPSG:4326',format='GTiff',resampleAlg='near', creationOptions = ['TFW=YES', 'COMPRESS=LZW'])#,copyMetadata=True)
        # filename='/home/sbf000/test.tif'
        if os.path.isfile(filename):
            os.remove(filename)

        rb = ds.GetRasterBand(1)
        driver = gdal.GetDriverByName('GTiff')
        display(rb.GetDataset().RasterCount)
        band = ds.GetRasterBand(1)
        width = ds.RasterXSize
        height = ds.RasterYSize
        display(width,height)
        data = band.ReadAsArray(0,0,width,height)
        display(data.shape)
        display(data.dtype)
        tods = driver.Create(filename, width, height, 1, options=["INTERLEAVE=PIXEL"])
        tods.WriteRaster(0,0, width, height, data.tobytes(), width, height, band_list=[1])
        tods.FlushCache()
        # src_ds = driver.CreateCopy(filename, rb.GetDataset(), 0 )

        # translate_options = gdal.TranslateOptions(format = 'GTiff', creationOptions = ['TFW=YES', 'COMPRESS=LZW'] )

        # ds = gdal.Warp(filename, src_ds, outputType=gdal.GDT_Float32, options=warp_options)
        ds = None
        print(f'Wrote {filename}')
        # if os.path.isfile(file):
        #     print(f'Erasing file {file}')
        #     os.remove(file)

        break
    
        

# from osgeo import osr
#     sr = osr.SpatialReference()
#     sr.ImportFromEPSG(4326)
#     ds.SetProjection(sr.ExportToWkt())




        
#         meta = rb.GetMetadata()

#         variables.append(f"{meta['GRIB_ELEMENT']},{meta['GRIB_COMMENT']},{meta['GRIB_SHORT_NAME']},{meta['GRIB_UNIT']},{rb.GetDescription()}\n")

# variables = np.sort(np.unique(variables))
# variables = np.insert(variables,0,'GRIB_ELEMENT,GRIB_COMMENT,GRIB_SHORT_NAME,GRIB_UNIT,DESCRIPTION\n')
# outfile = str(path / 'unique_vars.csv')

# if os.path.isfile(outfile):
#     os.remove(outfile)

# with open(outfile,'w+') as fp:
#     fp.writelines(variables)

In [ ]:
from osgeo import gdal, osr
import os, struct
import numpy as np


# layer = iface.activeLayer()

# provider = layer.dataProvider()

path = '/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/2020120700_000.grib2'

# fmttypes = {'Byte':'B', 'UInt16':'H', 'Int16':'h', 'UInt32':'I', 'Int32':'i', 'Float32':'f', 'Float64':'d'}

dataset = gdal.Open(path)

#Get projection
prj = dataset.GetProjection()

#setting number band (blue in this case)
number_band = 1

band = dataset.GetRasterBand(number_band)

desc = band.GetDescription()

no_data_val = band.GetNoDataValue()

meta = band.GetMetadata()

geotransform = dataset.GetGeoTransform()

# Set name of output raster
# if number_band == 3:
output_file = "/home/sbf000/ss6/t1.tif"
output_file_WGS84 = "/home/sbf000/ss6/t1WGS84.tif"

# Create gtif file with rows and columns from parent raster 
driver = gdal.GetDriverByName("GTiff")

columns, rows = (band.XSize, band.YSize)

# print ("rows = %d columns = %d" % (columns, rows))

BandType = gdal.GetDataTypeName(band.DataType)

# print ("Band Type = ", BandType)

# raster = []

# for y in range(band.YSize):

#     scanline = band.ReadRaster(0, y, band.XSize, 1, band.XSize, 1, band.DataType)
#     values = struct.unpack(fmttypes[BandType] * band.XSize, scanline)
#     raster.append(values)

raster = band.ReadAsArray(0,0,columns,rows)


dst_ds = driver.Create(output_file, 
                       columns, 
                       rows, 
                       1, 
                       gdal.GDT_Float32)

# #flattened list of raster values
# raster = [ item for element in raster for item in element ]

# #transforming list in array
# raster = np.asarray(np.reshape(raster, (rows,columns)))

##writting output raster
dst_ds.GetRasterBand(1).WriteArray( raster.astype(np.float32) )

#setting extension of output raster
# top left x, w-e pixel resolution, rotation, top left y, rotation, n-s pixel resolution
dst_ds.SetGeoTransform(geotransform)
dst_ds.GetRasterBand(1).SetDescription(desc)
dst_ds.GetRasterBand(1).SetNoDataValue(no_data_val)
dst_ds.GetRasterBand(1).SetMetadata(meta)
# setting spatial reference of output raster 
srs = osr.SpatialReference(wkt = prj)
dst_ds.SetProjection( srs.ExportToWkt() )
warp_options = gdal.WarpOptions(dstSRS='EPSG:4326',format='GTiff')#,resampleAlg='near')#, creationOptions = ['TFW=YES', 'COMPRESS=LZW'])#,copyMetadata=True)
ds = gdal.Warp(output_file_WGS84, dst_ds, outputType=gdal.GDT_Float32, options=warp_options)
ds = None
#Close output raster dataset 
dst_ds = None

#Close main raster dataset
dataset = None


In [ ]:
from osgeo import gdal, osr
from pathlib import Path
import copy
import datetime
import glob
import logging
import numpy as np
import re

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

def get_files_to_process(base_path:Path, target_date:str, target_times:list) -> list:
    # Make a list of files to be ingested
    grib_files = glob.glob(str(base_path / f'{target_date}*.grib2'))
    target_pos = len(str(base_path)) + 9
    filtered_grib_files = [f for f in grib_files if f[target_pos:target_pos+6] in target_times]
    return filtered_grib_files

def get_layer_name(description:str, metadata:dict, projection:str, band_num:int) -> str:
    extension = '.tif'
    level = copy.copy(description)
    type_of_level = copy.copy(description)
    level = re.sub('([0-9]+).*? .*?=.*$',r'\1',level)
    type_of_level = re.sub('.*? (.*?)=.*$',r'\1',type_of_level)
    if 'ISBL' == type_of_level:
        level = int(level)//100
        level = f'{level:04d}'

    layer_name = f"CMC_hrdps_continental_{metadata['GRIB_ELEMENT']}_{type_of_level}_{level}_{projection}_{datetime.datetime.fromtimestamp(int(metadata['GRIB_VALID_TIME'])).strftime('%Y%m%d%H')}_P{int(metadata['GRIB_FORECAST_SECONDS'])//3600:03d}-00_{band_num}{extension}"
    layer_name = layer_name.replace(' ','')
    return layer_name

def create_dataset_from_band(driver:gdal.Driver, output_file_name: str, dataset_info:dict, band_info:dict) -> gdal.Dataset:
        # create a dataset for that band
        dst_ds = driver.Create(output_file_name, 
                            band_info['xsize'], 
                            band_info['ysize'], 
                            1, 
                            gdal.GDT_Float32)

        # write output raster
        dst_ds.GetRasterBand(1).WriteArray(band_info['raster_array'].astype(np.float32))
        # set geotransform
        dst_ds.SetGeoTransform(dataset_info['geotransform'])
        # set description
        dst_ds.GetRasterBand(1).SetDescription(band_info['description'])
        #set no data value
        dst_ds.GetRasterBand(1).SetNoDataValue(band_info['no_data_val'])
        # set metadata
        dst_ds.GetRasterBand(1).SetMetadata(band_info['metadata'])
        # set spatial reference of output raster 
        srs = osr.SpatialReference(wkt = dataset_info['projection'])
        dst_ds.SetProjection( srs.ExportToWkt() )        
        return dst_ds

def warp_to_wgs84(output_file_name:str, dst_ds:gdal.Dataset) -> gdal.Dataset:

    logger.info(f'Reprojecting {output_file_name}')

    warp_options = gdal.WarpOptions(dstSRS='EPSG:4326',format='GTiff',resampleAlg='near')#, creationOptions = ['TFW=YES', 'COMPRESS=LZW'])#,copyMetadata=True)
    
    warped_ds = gdal.Warp(output_file_name, dst_ds, outputType=gdal.GDT_Float32, options=warp_options)

    return warped_ds

def process_band(band_number:int, base_path:Path, band:gdal.Band, dataset_info:dict, reproject:bool=False) -> gdal.Dataset:
        band_info = {}
        # get band description
        band_info['description'] = band.GetDescription()
        # get band no data value
        band_info['no_data_val'] = 9999
        # get band metadata
        band_info['metadata'] = band.GetMetadata()
        # get band data
        band_info['raster_array'] = band.ReadAsArray(0,0,band.XSize,band.YSize)
        # get band size
        band_info['xsize'] = band.XSize
        band_info['ysize'] = band.YSize

        layer_name = get_layer_name(band_info['description'], band_info['metadata'], 'ps2.5km', band_number)

        print(f'Processing {layer_name}')
        output_file_name = str(base_path / layer_name)

        dst_ds = create_dataset_from_band(driver, output_file_name, dataset_info, band_info)

        if reproject:
            layer_name = get_layer_name(band_info['description'], band_info['metadata'], 'EPSG_4326', band_number)
            output_file_name = str(base_path / layer_name)
            warped_ds = warp_to_wgs84(output_file_name, band_info, dst_ds)
            return warped_ds

        return dst_ds
            
        
def process_grib_file(base_path:Path, grib_file:str) -> None:
    logger.info(f'Processing {grib_file}')
    # open
    dataset = gdal.Open(grib_file)

    dataset_info = {}
    # get projection
    dataset_info['projection'] = dataset.GetProjection()

    # get geotransform
    dataset_info['geotransform'] = dataset.GetGeoTransform()
    
    logger.info(f'Number of bands to process {dataset.RasterCount}')
    # for each band
    for i in range(1,dataset.RasterCount+1):
        # get current band
        band = dataset.GetRasterBand(i)
        
        dst_ds = process_band(i, base_path, band, dataset_info, reproject=False)
    
        # cleanup 
        del dst_ds    

        # cleanup
        del band

    # cleanup
    del dataset


base_path = Path('/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/')

target_date = '20170101'

targets_24h = ['00_000','00_024','00_048','06_018','06_42','12_012','12_036','18_006','18_030']

driver = gdal.GetDriverByName("GTiff")

filtered_grib_files = get_files_to_process(base_path, target_date, targets_24h)

# for each file
for grib_file in filtered_grib_files[0:1]:
    process_grib_file(base_path, grib_file)



In [26]:
from osgeo import gdal, osr
from pathlib import Path
import copy
import datetime
import glob
import logging
import numpy as np
import re
import pandas as pd
import multiprocessing

level_types = {'DBLL':1, 'EATM':2, 'EtaL':3, 'HTGL':4, 'ISBL':5, 'MSL':6, 'NTAT':7, 'SFC':8}
base_path = Path('/fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/')
target_date = '20200101'
grib_files = glob.glob(str(base_path / f'{target_date}*.grib2'))


def process_band(dataset,i):
    # get current band
    band = dataset.GetRasterBand(i)
    description = band.GetDescription()
    # print(description)
    level = copy.copy(description)
    # print(level)
    type_of_level = copy.copy(description)
    level_unit = copy.copy(description)
    level = re.sub('(.*?)\[.*? .*?=.*$',r'\1',level)
    if '-' in level:
        level = '@'.join(level.split('-'))

    # level  = 
    # print(level)
    type_of_level_name = re.sub('.*? (.*?)=.*$',r'\1',type_of_level).strip()
    # print(type_of_level_name)
    type_of_level_description = re.sub('.*? .*?=(.*$)',r'\1',type_of_level)
    # print(type_of_level_description)
    level_unit = re.sub('(.*?) .*?=.*$',r'\1',level_unit)
    level_unit = level_unit.replace(level,'')
    level_unit = re.sub('.*\[(.*?).$',r'\1',level_unit)
    level_unit = level_unit.replace('-','').strip()
    type_of_level_description = type_of_level_description.replace('"','').strip()

    meta = band.GetMetadata()
    mydict = {}
    # if '' == level_unit:
    #     desc_long = ' - '.join([meta['GRIB_COMMENT'].replace(meta['GRIB_UNIT'],''), type_of_level_description])
    # else:
    #     desc_long = ' - '.join([meta['GRIB_COMMENT'].replace(meta['GRIB_UNIT'],''), type_of_level_description, level_unit])


    mydict['name'] = meta['GRIB_COMMENT'].strip().replace(meta['GRIB_UNIT'],'').strip()
    mydict['name_alternate'] = meta['GRIB_ELEMENT'].strip()
    mydict['description_short'] = meta['GRIB_COMMENT'].strip().replace(meta['GRIB_UNIT'],'').strip()
    # mydict['description_long'] = desc_long 
    mydict['datatype'] = 'fl'
    mydict['units'] = meta['GRIB_UNIT'].replace('[','').replace(']','')
    mydict['status'] = 'ACTIVE'
    mydict['level'] = 15
    mydict['interpolation'] = 'near'
    mydict['type_of_level'] = type_of_level_name
    mydict['type_of_level_description'] = type_of_level_description
    # mydict['level'] = level
    mydict['level_units'] = level_unit
    # mydict['description'] = description

    # names.append(meta['GRIB_ELEMENT'])
    # descriptions.append(meta['GRIB_COMMENT'])
    # units.append(meta['GRIB_UNIT'])
    # type_of_level_names.append(type_of_level_name)
    # type_of_level_descriptions.append(type_of_level_description)
    # levels.append(level)
    shared_list.append(mydict)


# {"name": "l4ftx", "name_alternate": "l4ftx", "description_short": "Best lifted index", "datatype": "fl", "units": "C", "status": "ACTIVE", "level": 15, "interpolation": "near"} 
# for f in grib_files:
#         dataset = gdal.Open(f)
#         for i in range(1,dataset.RasterCount+1):
#             process_band(dataset,i)

def process_file(f):
    print(f'Processing {f}')
    dataset = gdal.Open(f)
    for i in range(1,dataset.RasterCount+1):
        process_band(dataset,i)
    

print(f'Processing {len(grib_files)} files\n')
# # for f in grib_files:
# #     process_file(f)


manager = multiprocessing.Manager()
shared_list = manager.list()
with multiprocessing.Pool(25) as p:
        p.map(process_file, grib_files)

print(f'Finished processing {len(grib_files)} files\n')


# df = pd.DataFrame({'name':names,'description':descriptions,'unit':units, 'type_of_level':type_of_level_names, 'type_of_level_description':type_of_level_descriptions, 'level': levels, 'level_units':level_units})
df = pd.DataFrame(list(shared_list))

df = df.drop_duplicates().sort_values('name')


# groups = df.groupby(['name', 'unit', 'type_of_level', 'level_units'])
# df_list = []
# for _,cur_df in groups:
#     # display(cur_df)
#     if len(cur_df.index) > 1:
#         levels = ','.join(cur_df.level.tolist())
#         new_df = pd.DataFrame([cur_df.iloc[0].to_dict()])
#         new_df['level'] = levels
#         cur_df = copy.deepcopy(new_df)
#     df_list.append(cur_df)


# df = pd.concat(df_list,ignore_index=True).sort_values('name')
# df['id'] = np.arange(len(df.index)) + 1

# df
groups = df.groupby('type_of_level')
for type_of_level, sub_df in groups:
    sub_df['id'] = np.arange(len(sub_df.index)) + 1
    csv_file = f'../data_layers_to_ingest_{type_of_level}.csv' 
    if os.path.isfile(csv_file):
        os.remove(csv_file)
    sub_df[['name','name_alternate','description_short','datatype','units','status','level','interpolation','id']].to_csv(csv_file,index=False)
    ndf = pd.read_csv(csv_file)
    pd.options.display.max_rows = 999
    display(type_of_level)
    display({a:b for a,b in zip(sub_df.name_alternate.to_list(),sub_df.id.to_list())})
    display(sub_df)



Processing 11 files

Processing /fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/2020010118_006.grib2Processing /fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/2020010112_012.grib2Processing /fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/2020010118_042.grib2Processing /fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/2020010100_048.grib2Processing /fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/2020010100_024.grib2Processing /fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/2020010106_030.grib2Processing /fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/2020010112_036.grib2Processing /fs/site6/eccc/cmd/w/sbf000/operation.forecasts.lamgrib2.hrdps_national.continental/2020010100_000.grib2Processing /fs/site6/eccc/cmd/w/sbf000/operation.forecasts.

'DBLL'

{'SOILM': 1, 'TSOIL': 2}

,name,name_alternate,description_short,datatype,units,status,level,interpolation,type_of_level,type_of_level_description,level_units,id
147,Soil moisture content,SOILM,Soil moisture content,fl,kg/(m^2),ACTIVE,15,near,DBLL,Depth below land surface,m,1
141,Soil temperature,TSOIL,Soil temperature,fl,C,ACTIVE,15,near,DBLL,Depth below land surface,m,2


'EATM'

...

{'DSWRF': 1, 'ULWRF': 2, 'USWRF': 3}

,name,name_alternate,description_short,datatype,units,status,level,interpolation,type_of_level,type_of_level_description,level_units,id
12,Downward short-wave radiation flux,DSWRF,Downward short-wave radiation flux,fl,W/(m^2),ACTIVE,15,near,NTAT,Nominal top of atmosphere,,1
30,Upward long-wave radiation flux,ULWRF,Upward long-wave radiation flux,fl,W/(m^2),ACTIVE,15,near,NTAT,Nominal top of atmosphere,,2
29,Upward short-wave radiation flux,USWRF,Upward short-wave radiation flux,fl,W/(m^2),ACTIVE,15,near,NTAT,Nominal top of atmosphere,,3


'SFC'

{'APCP06': 1,
 'APCP12': 2,
 'APCP18': 3,
 'APCP24': 4,
 'APCP30': 5,
 'APCP36': 6,
 'APCP42': 7,
 'APCP48': 8,
 'ALBDO': 9,
 'ACPCP': 10,
 'DLWRF': 11,
 'DSWRF': 12,
 'FPRATE': 13,
 'HGT': 14,
 'ICEC': 15,
 'IPRATE': 16,
 'LAND': 17,
 'LHTFL': 18,
 'NLWRS': 19,
 'NSWRS': 20,
 'HPBL': 21,
 'PRATE': 22,
 'PTYPE': 23,
 'PRES': 24,
 'RPRATE': 25,
 'SHTFL': 26,
 'SKINT': 27,
 'SDEN': 28,
 'SNOD': 29,
 'SDWE': 30,
 'SPRATE': 31,
 'SOILVIC': 32,
 'TSOIL': 33,
 'SPFH': 34,
 'SFCWRO': 35,
 'TCDC': 36,
 'WTMP': 37}

,name,name_alternate,description_short,datatype,units,status,level,interpolation,type_of_level,type_of_level_description,level_units,id
36,06 hr Total precipitation,APCP06,06 hr Total precipitation,fl,kg/(m^2),ACTIVE,15,near,SFC,Ground or water surface,,1
342,12 hr Total precipitation,APCP12,12 hr Total precipitation,fl,kg/(m^2),ACTIVE,15,near,SFC,Ground or water surface,,2
471,18 hr Total precipitation,APCP18,18 hr Total precipitation,fl,kg/(m^2),ACTIVE,15,near,SFC,Ground or water surface,,3
64,24 hr Total precipitation,APCP24,24 hr Total precipitation,fl,kg/(m^2),ACTIVE,15,near,SFC,Ground or water surface,,4
10,30 hr Total precipitation,APCP30,30 hr Total precipitation,fl,kg/(m^2),ACTIVE,15,near,SFC,Ground or water surface,,5
398,36 hr Total precipitation,APCP36,36 hr Total precipitation,fl,kg/(m^2),ACTIVE,15,near,SFC,Ground or water surface,,6
157,42 hr Total precipitation,APCP42,42 hr Total precipitation,fl,kg/(m^2),ACTIVE,15,near,SFC,Ground or water surface,,7
262,48 hr Total precipitation,APCP48,48 hr Total precipitation,fl,kg/(m^2),ACTIVE,15,near,SFC,Ground or water surface,,8
136,Albedo,ALBDO,Albedo,fl,%,ACTIVE,15,near,SFC,Ground or water surface,,9
11,Convective precipitation,ACPCP,Convective precipitation,fl,kg/(m^2),ACTIVE,15,near,SFC,Ground or water surface,,10


In [19]:
import json
import pandas as pd
import numpy as np
import glob

csv_files = glob.glob('../data_layers_to_ingest_*.csv')
for csv_file in csv_files:
    ndf = pd.read_csv(csv_file)
    xls_file = ''.join(['../',str(Path(csv_file).stem), '.xlsx'])
    if os.path.isfile(xls_file):
        os.remove(xls_file)
    ndf.to_excel(xls_file,index=False)

    result = ndf.to_json(orient="records")
    parsed = json.loads(result)
    json_text = json.dumps(parsed, indent=4)
    json_file = ''.join(['../',str(Path(csv_file).stem), '.json'])
    if os.path.isfile(json_file):
        os.remove(json_file)
    with open(json_file,'w') as fp:
        fp.write(json_text)

# pd.options.display.max_rows = 999
# # cols = list(ndf.columns)
# # cols.remove('')
# tmp_df = pd.DataFrame({'name':ndf.name.unique()})
# tmp_df['id'] = np.arange(len(tmp_df.index)) + 1

# # tmp_df
# final_df = ndf.merge(tmp_df,on='name')

# display(final_df)

# result = final_df.to_json(orient="records")
# parsed = json.loads(result)
# json_text = json.dumps(parsed, indent=4)
# with open('../data_layers.json','w') as fp:
#     fp.write(json_text)
